In [2]:
import pandas as pd
from rectools import Columns
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset
from rectools.models import PopularModel, PureSVDModel
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

/home/bazilyq/Рабочий стол/work/RecTools/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
np.random.seed(0)
DATA_PATH = '../datasets/KION/data/'
users_df = pd.read_csv(DATA_PATH + 'users.csv',)
items_df = pd.read_csv(DATA_PATH + 'items.csv',)
interactions = pd.read_csv(DATA_PATH + 'interactions.csv')

# interactions_train = interactions.query("last_watch_dt >= '2021-07-15' and last_watch_dt < '2021-08-16'").copy()
# interactions_train = interactions.query("last_watch_dt >= '2021-08-10' and last_watch_dt < '2021-08-16'").copy()
interactions_train = interactions.query("last_watch_dt >= '2021-07-15' and last_watch_dt < '2021-08-16'").copy()
# (9646, 3273)
interactions_test = interactions.query("last_watch_dt >= '2021-08-16'").copy()

sample_users = np.random.choice(interactions_train['user_id'].unique(), 1000)
interactions_train = interactions_train[interactions_train['user_id'].isin(sample_users)]

interactions_test = interactions_test.loc[
    interactions_test["user_id"].isin(interactions_train["user_id"])
    & interactions_test["item_id"].isin(interactions_train["item_id"])
]

interactions_train.fillna({"watched_pct": 1}, inplace=True)
interactions_train['watched_pct'] = interactions_train['watched_pct'].astype(int)

dataset = Dataset.construct(
    interactions_df=interactions_train
    .rename(
        columns={
            "user_id": Columns.User,
            "item_id": Columns.Item,
            "last_watch_dt": Columns.Datetime,
            "watched_pct": Columns.Weight,
        }
    )
    .reindex(columns=Columns.Interactions)
)

test_users = interactions_test["user_id"].unique()
test_users.size

user_ext_to_int_map = dataset.user_id_map.to_internal.to_dict()
item_int_to_ext_map = dataset.item_id_map.to_external.to_dict()

ui_csr = dataset.get_user_item_matrix()
ui_csr.shape

(997, 1438)

In [4]:
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=20, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

iteration: 1, train RMSE: 35.600951958687276
iteration: 2, train RMSE: 22.605953240465332
iteration: 3, train RMSE: 16.410673982659965
iteration: 4, train RMSE: 12.96070450345643
iteration: 5, train RMSE: 10.84119243821546
iteration: 6, train RMSE: 9.493806481368287
iteration: 7, train RMSE: 8.740212965322337
iteration: 8, train RMSE: 8.308433696459307
iteration: 9, train RMSE: 8.047001377090385
iteration: 10, train RMSE: 7.980250005138647
iteration: 11, train RMSE: 7.950013461893248
iteration: 12, train RMSE: 8.015180060981162
iteration: 13, train RMSE: 8.143045590422387
iteration: 14, train RMSE: 8.294917319489405
iteration: 15, train RMSE: 8.437511356805063
iteration: 16, train RMSE: 8.501827556762388
iteration: 17, train RMSE: 8.61754297027165
iteration: 18, train RMSE: 8.790811171048066
iteration: 19, train RMSE: 8.911039399763153
iteration: 20, train RMSE: 9.065672687974677
precision: 0.0029126213592233015
recall: 0.011650485436893204
(206,) (398,)


In [7]:
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=20, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

iteration: 1, train RMSE: 29.511610105457418
iteration: 2, train RMSE: 20.577874176492877
iteration: 3, train RMSE: 16.883849166245387
iteration: 4, train RMSE: 14.970765433845147
iteration: 5, train RMSE: 13.835454943044821
iteration: 6, train RMSE: 13.093516702980299
iteration: 7, train RMSE: 12.574120754855697
iteration: 8, train RMSE: 12.191131194644743
iteration: 9, train RMSE: 11.897390043462655
iteration: 10, train RMSE: 11.665619152878032
iteration: 11, train RMSE: 11.474864713901207
iteration: 12, train RMSE: 11.317848379816517
iteration: 13, train RMSE: 11.18396395939861
iteration: 14, train RMSE: 11.069361260133444
iteration: 15, train RMSE: 10.970702947557829
iteration: 16, train RMSE: 10.883415449071185
iteration: 17, train RMSE: 10.805414014811705
iteration: 18, train RMSE: 10.737164165728798
iteration: 19, train RMSE: 10.675300709458448
iteration: 20, train RMSE: 10.6185440342381
precision: 0.0001463907117617365
recall: 0.000314029136579675
(19810,) (897,)


In [8]:
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=30, n_feature=20, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

iteration: 1, train RMSE: 27.042942275519078
iteration: 2, train RMSE: 16.36048312548672
iteration: 3, train RMSE: 12.134469263751907
iteration: 4, train RMSE: 9.928110233608209
iteration: 5, train RMSE: 8.618490795468169
iteration: 6, train RMSE: 7.78364385443915
iteration: 7, train RMSE: 7.222086914379448
iteration: 8, train RMSE: 6.832077948351645
iteration: 9, train RMSE: 6.553814572453235
iteration: 10, train RMSE: 6.34849079597141
iteration: 11, train RMSE: 6.195956935483202
iteration: 12, train RMSE: 6.078028457432925
iteration: 13, train RMSE: 5.987387383473297
iteration: 14, train RMSE: 5.91409254029425
iteration: 15, train RMSE: 5.857732081443643
iteration: 16, train RMSE: 5.813297609655142
iteration: 17, train RMSE: 5.776826576374663
iteration: 18, train RMSE: 5.747169880476732
iteration: 19, train RMSE: 5.723884839612709
iteration: 20, train RMSE: 5.706124342645071
iteration: 21, train RMSE: 5.693482871788108
iteration: 22, train RMSE: 5.682286786947262
iteration: 23, train

In [3]:
pure_svd_model = PopularModel(verbose=1)
pure_svd_model.fit(dataset)

recs = pure_svd_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print('PopularModel')
print(f"precision: {precision_value}\nrecall: {recall_value}", '\n')

100%|██████████| 3716/3716 [00:00<00:00, 152433.63it/s]

PopularModel
precision: 0.03789020452099032
recall: 0.16584749259909268 



In [5]:
pure_svd_model = PureSVDModel(verbose=1)
pure_svd_model.fit(dataset)

recs = pure_svd_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print('PureSVDModel')
print(f"precision: {precision_value}\nrecall: {recall_value}", '\n')
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

PureSVDModel
precision: 0.016442411194833154
recall: 0.06239342800454374 

(3716,) (112,)


In [1]:
# # добавлены гиперпараметры в конструктор
# from rectools.models import BayesProbMF_V2

# T, D, initial_cutoff, lowest_rating, highest_rating = 3, 10, 0, 0, 100
# bpmf_model = BayesProbMF_V2(verbose=1, T=T, D=D, initial_cutoff=initial_cutoff, lowest_rating=lowest_rating, highest_rating=highest_rating)
# bpmf_model.fit(dataset)

# recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

# precision = Precision(k=10)
# recall = Recall(k=10)

# precision_value = precision.calc(reco=recs, interactions=interactions_test)
# recall_value = recall.calc(reco=recs, interactions=interactions_test)
# print(f"precision: {precision_value}\nrecall: {recall_value}")

In [8]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

0
iteration: 1, train RMSE: 32.63250719505038
1
iteration: 2, train RMSE: 20.975747831859152
2
iteration: 3, train RMSE: 15.3575838802549
3
iteration: 4, train RMSE: 12.221639801814241
4
iteration: 5, train RMSE: 10.344854840825892
5
iteration: 6, train RMSE: 9.178165858001162
6
iteration: 7, train RMSE: 8.463375237254349
7
iteration: 8, train RMSE: 8.02376023211319
8
iteration: 9, train RMSE: 7.74766681126756
9
iteration: 10, train RMSE: 7.629712043753078
precision: 0.004790096878363832
recall: 0.018530246610991296


In [9]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=1.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

0
iteration: 1, train RMSE: 35.308919376284116
1
iteration: 2, train RMSE: 27.500595515711737
2
iteration: 3, train RMSE: 21.224229555684182
3
iteration: 4, train RMSE: 17.18039744988234
4
iteration: 5, train RMSE: 14.485652850553663
5
iteration: 6, train RMSE: 12.631019976048803
6
iteration: 7, train RMSE: 11.316196737176112
7
iteration: 8, train RMSE: 10.366950901158669
8
iteration: 9, train RMSE: 9.678874616704261
9
iteration: 10, train RMSE: 9.203263346366159
precision: 0.0015608180839612488
recall: 0.0059296736439299605


In [10]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2.2)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

0
iteration: 1, train RMSE: 32.16026481729534
1
iteration: 2, train RMSE: 20.14693902464928
2
iteration: 3, train RMSE: 14.678100737929086
3
iteration: 4, train RMSE: 11.67148959356015
4
iteration: 5, train RMSE: 9.899269947033066
5
iteration: 6, train RMSE: 8.814875873294312
6
iteration: 7, train RMSE: 8.178758331706927
7
iteration: 8, train RMSE: 7.80730213623482
8
iteration: 9, train RMSE: 7.607312702657746
9
iteration: 10, train RMSE: 7.579424454481529
precision: 0.0035522066738428427
recall: 0.013622930572093177


In [9]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=1.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

0
iteration: 1, train RMSE: 32.63643061584835
1
iteration: 2, train RMSE: 20.977520648618935
2
iteration: 3, train RMSE: 15.358692204617569
3
iteration: 4, train RMSE: 12.222319090980443
4
iteration: 5, train RMSE: 10.345195727230491
5
iteration: 6, train RMSE: 9.17910099078167
6
iteration: 7, train RMSE: 8.46346946543157
7
iteration: 8, train RMSE: 8.025458450130047
8
iteration: 9, train RMSE: 7.754780434115413
9
iteration: 10, train RMSE: 7.638859266392929
precision: 0.005059203444564048
recall: 0.019409706814137652


In [10]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_item=1.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

0
iteration: 1, train RMSE: 32.64891864474101
1
iteration: 2, train RMSE: 20.984261781122452
2
iteration: 3, train RMSE: 15.363286467441263
3
iteration: 4, train RMSE: 12.225675878795629
4
iteration: 5, train RMSE: 10.348125114584198
5
iteration: 6, train RMSE: 9.177535977923805
6
iteration: 7, train RMSE: 8.466390222408556
7
iteration: 8, train RMSE: 8.029596722635674
8
iteration: 9, train RMSE: 7.752473033282968
9
iteration: 10, train RMSE: 7.6431118244884875
precision: 0.00476318622174381
recall: 0.01797356070546841


In [11]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=1., beta_item=1.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

0
iteration: 1, train RMSE: 32.6528414471397
1
iteration: 2, train RMSE: 20.98603896397132
2
iteration: 3, train RMSE: 15.364408550399405
3
iteration: 4, train RMSE: 12.226379768478088
4
iteration: 5, train RMSE: 10.348588345787574
5
iteration: 6, train RMSE: 9.176700265114642
6
iteration: 7, train RMSE: 8.463590478693886
7
iteration: 8, train RMSE: 8.023022442534746
8
iteration: 9, train RMSE: 7.748493074490809
9
iteration: 10, train RMSE: 7.642072223849097
precision: 0.004843918191603875
recall: 0.018478634871383317


In [13]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=25, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=1., beta_item=1.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

0
iteration: 1, train RMSE: 32.6528414471397
1
iteration: 2, train RMSE: 20.98603896397132
2
iteration: 3, train RMSE: 15.364408550399405
3
iteration: 4, train RMSE: 12.226379768478088
4
iteration: 5, train RMSE: 10.348588345787574
5
iteration: 6, train RMSE: 9.176700265114642
6
iteration: 7, train RMSE: 8.463590478693886
7
iteration: 8, train RMSE: 8.023022442534746
8
iteration: 9, train RMSE: 7.748493074490809
9
iteration: 10, train RMSE: 7.642072223849097
10
iteration: 11, train RMSE: 7.633995354892715
11
iteration: 12, train RMSE: 7.669829441069797
12
iteration: 13, train RMSE: 7.732383034828779
13
iteration: 14, train RMSE: 7.843592927469596
14
iteration: 15, train RMSE: 7.945400898654401
15
iteration: 16, train RMSE: 8.073667403215412
16
iteration: 17, train RMSE: 8.21308817265869
17
iteration: 18, train RMSE: 8.331650061033008
18
iteration: 19, train RMSE: 8.439179945357509
19
iteration: 20, train RMSE: 8.539930188128377
20
iteration: 21, train RMSE: 8.643861706026847
21
iterati

In [10]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

T, D, initial_cutoff, lowest_rating, highest_rating = 5, 10, 0, 0, 100
bpmf_model = BayesProbMF_V2(verbose=1, T=T, D=D, initial_cutoff=initial_cutoff, lowest_rating=lowest_rating, highest_rating=highest_rating)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

9646 3273
9646 3273
0
1
2
3
4
precision: 0.003659849300322928
recall: 0.013941019110395868


In [11]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

T, D, initial_cutoff, lowest_rating, highest_rating = 10, 10, 0, 0, 100
bpmf_model = BayesProbMF_V2(verbose=1, T=T, D=D, initial_cutoff=initial_cutoff, lowest_rating=lowest_rating, highest_rating=highest_rating)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

9646 3273
9646 3273
0
1
2
3
4
5
6
7
8
9
precision: 0.004790096878363832
recall: 0.018530246610991296


In [8]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

T, D, initial_cutoff, lowest_rating, highest_rating = 10, 20, 0, 0, 100
bpmf_model = BayesProbMF_V2(verbose=1, T=T, D=D, initial_cutoff=initial_cutoff, lowest_rating=lowest_rating, highest_rating=highest_rating)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

9646 3273
9646 3273
0
iteration: %d, train RMSE: %.6f 1 31.425092745091295
1
iteration: %d, train RMSE: %.6f 2 19.18725881150648
2
iteration: %d, train RMSE: %.6f 3 13.950517255640342
3
iteration: %d, train RMSE: %.6f 4 11.165711969061142
4
iteration: %d, train RMSE: %.6f 5 9.515639612871112
5
iteration: %d, train RMSE: %.6f 6 8.46632836114417
6
iteration: %d, train RMSE: %.6f 7 7.84109699836319
7
iteration: %d, train RMSE: %.6f 8 7.482268848657962
8
iteration: %d, train RMSE: %.6f 9 7.284678970206231
9
iteration: %d, train RMSE: %.6f 10 7.206825565625835
precision: 0.0021797631862217437
recall: 0.007743531223433212


In [9]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

T, D, initial_cutoff, lowest_rating, highest_rating = 10, 10, 0, 0, 100
bpmf_model = BayesProbMF_V2(verbose=1, T=T, D=D, initial_cutoff=initial_cutoff, lowest_rating=lowest_rating, highest_rating=highest_rating)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

9646 3273
9646 3273
0
iteration: %d, train RMSE: %.6f 1 32.63250719505038
1
iteration: %d, train RMSE: %.6f 2 20.975747831859152
2
iteration: %d, train RMSE: %.6f 3 15.3575838802549
3
iteration: %d, train RMSE: %.6f 4 12.221639801814241
4
iteration: %d, train RMSE: %.6f 5 10.344854840825892
5
iteration: %d, train RMSE: %.6f 6 9.178165858001162
6
iteration: %d, train RMSE: %.6f 7 8.463375237254349
7
iteration: %d, train RMSE: %.6f 8 8.02376023211319
8
iteration: %d, train RMSE: %.6f 9 7.74766681126756
9
iteration: %d, train RMSE: %.6f 10 7.629712043753078
precision: 0.004790096878363832
recall: 0.018530246610991296


In [13]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

T, D, initial_cutoff, lowest_rating, highest_rating = 10, 10, 0, 0, 100
bpmf_model = BayesProbMF_V2(verbose=1, T=T, D=D, initial_cutoff=initial_cutoff, lowest_rating=lowest_rating, highest_rating=highest_rating)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

9646 3273
9646 3273
0
iteration: %d, train RMSE: %.6f 1 32.63250719505038
1
iteration: %d, train RMSE: %.6f 2 20.975747831859152
2
iteration: %d, train RMSE: %.6f 3 15.3575838802549
3
iteration: %d, train RMSE: %.6f 4 12.221639801814241
4
iteration: %d, train RMSE: %.6f 5 10.344854840825892
5
iteration: %d, train RMSE: %.6f 6 9.178165858001162
6
iteration: %d, train RMSE: %.6f 7 8.463375237254349
7
iteration: %d, train RMSE: %.6f 8 8.02376023211319
8
iteration: %d, train RMSE: %.6f 9 7.74766681126756
9
iteration: %d, train RMSE: %.6f 10 7.629712043753078
precision: 0.004790096878363832
recall: 0.018530246610991296


In [15]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

T, D, initial_cutoff, lowest_rating, highest_rating = 25, 10, 0, 0, 100
bpmf_model = BayesProbMF_V2(verbose=1, T=T, D=D, initial_cutoff=initial_cutoff, lowest_rating=lowest_rating, highest_rating=highest_rating)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

9646 3273
9646 3273
0
iteration: %d, train RMSE: %.6f 1 32.63250719505038
1
iteration: %d, train RMSE: %.6f 2 20.975747831859152
2
iteration: %d, train RMSE: %.6f 3 15.3575838802549
3
iteration: %d, train RMSE: %.6f 4 12.221639801814241
4
iteration: %d, train RMSE: %.6f 5 10.344854840825892
5
iteration: %d, train RMSE: %.6f 6 9.178165858001162
6
iteration: %d, train RMSE: %.6f 7 8.463375237254349
7
iteration: %d, train RMSE: %.6f 8 8.02376023211319
8
iteration: %d, train RMSE: %.6f 9 7.74766681126756
9
iteration: %d, train RMSE: %.6f 10 7.629712043753078
10
iteration: %d, train RMSE: %.6f 11 7.6107269054894635
11
iteration: %d, train RMSE: %.6f 12 7.651304771948132
12
iteration: %d, train RMSE: %.6f 13 7.716848942060951
13
iteration: %d, train RMSE: %.6f 14 7.818895957627809
14
iteration: %d, train RMSE: %.6f 15 7.914735881130123
15
iteration: %d, train RMSE: %.6f 16 8.047061437714122
16
iteration: %d, train RMSE: %.6f 17 8.19467128605122
17
iteration: %d, train RMSE: %.6f 18 8.3074391

In [ ]:
# добавлены гиперпараметры в конструктор
from rectools.models import BayesProbMF_V2

T, D, initial_cutoff, lowest_rating, highest_rating = 50, 10, 0, 0, 100
bpmf_model = BayesProbMF_V2(verbose=1, T=T, D=D, initial_cutoff=initial_cutoff, lowest_rating=lowest_rating, highest_rating=highest_rating)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")

9646 3273
9646 3273
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
precision: 0.0031754574811625404
recall: 0.011929837609850235


In [ ]:
recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

precision: 0.0031754574811625404
recall: 0.011929837609850235
(3716,) (1371,)
